In [1]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 913 kB/s 
     |████████████████████████████████| 3.1 MB 1.8 MB/s 
     |████████████████████████████████| 61 kB 338 kB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=ed4bed5b133dae265281ea326437b9b0e0c6ae90a27a80782faf64ef3ad9e93b
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
  Attempting uninstall: transformers
    Found existing installation: transformers 4.5.1
    Uninstalling transformers-4.5.1:
      Successfully uninstalled transformers-4.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source

In [2]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from collections import Counter

import gc
import cloudpickle

import os, sys
import random

import warnings

import xgboost as xgb


import base64
import io

import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, util
from PIL import Image

from tqdm import tqdm

print('Using \033[34mXGBoost', xgb.__version__, '\033[0m')

Using XGBoost 1.5.0 


In [3]:
model = SentenceTransformer('clip-ViT-B-32')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/605M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/389 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/961k [00:00<?, ?B/s]

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


In [4]:
def transform2image_embeddings(df):
    image2text_embeddings = []
    widths = []
    heights = []

    for k in tqdm(range(df.shape[0])):
        data = df.loc[k, 1]
        
        try:
            image = base64.b64decode(data)
            img = Image.open(io.BytesIO(image)).convert('RGB')
        
            width, height = img.size
            widths.append(width)
            heights.append(height)

            image2text_embeddings.append(model.encode(img, show_progress_bar=False))
        except:
            print(k)
            widths.append(np.nan)
            heights.append(np.nan)
            image2text_embeddings.append(np.full(512, np.nan))
            
        
    image2text_embeddings = np.vstack(image2text_embeddings)
    
    embeddings_df = pd.DataFrame(image2text_embeddings)
    embeddings_df.insert(loc=0, column='image_id', value=df['image_id'])
    
    size_df = pd.DataFrame({'image_id': df['image_id'], 'width': widths, 'height': heights})
    
    return embeddings_df, size_df

In [5]:
import requests


def prepare_train_embeddings(images_path, indices):
    images = pd.read_csv(images_path)
    images['image_id'] = images.index
    
    container = []
    sizes = []
    
    found = 0
    progress_bar = indices

    for index in progress_bar:
        print('\033[34mHandling part {}...\033[0m'.format(index))
        url = 'https://analytics.wikimedia.org/published/datasets/one-off/caption_competition/training/image_pixels/part-00{0:03d}-04b253b8-db8c-4d14-a23f-3433a86841b4-c000.csv.gz'.format(index)
        r = requests.get(url, allow_redirects=True)

        with open('pixels.csv', 'wb') as file:
            file.write(r.content)

        df = pd.read_csv('pixels.csv', compression='gzip', sep='\t', header=None)
        df.columns = ['image_url', 1, 2]
        df = pd.merge(images, df, on='image_url')
        df, size_df = transform2image_embeddings(df)
        
        gc.collect()
        container.append(df)
        sizes.append(size_df)
        
        found += df.shape[0]
#         progress_bar.set_description('Found: {0:.2f}'.format(found / images.shape[0]))

        print('found: {0:.2f}'.format(found / images.shape[0]))
        print()
        
        if found == images.shape[0]:
            break
    
    container = pd.concat(container)
    sizes = pd.concat(sizes)
    
    temp = images['image_id']
    
    container = pd.merge(temp, container, on='image_id', how='left').sort_values(by='image_id')
    container.fillna(np.float32(1.), inplace=True)
    
    sizes = pd.merge(temp, sizes, on='image_id', how='left').sort_values(by='image_id')

    return container, sizes

In [6]:
%%time

embeddings, sizes = prepare_train_embeddings('../input/final-image-traindataset-part0-4-count-5/final_images.csv', range(200))

Handling part 0...


100%|██████████| 317/317 [01:02<00:00,  5.10it/s]


found: 0.00

Handling part 1...


100%|██████████| 331/331 [01:03<00:00,  5.17it/s]


found: 0.01

Handling part 2...


...

found: 0.93

Handling part 198...


100%|██████████| 344/344 [01:03<00:00,  5.39it/s]


found: 0.94

Handling part 199...


100%|██████████| 326/326 [01:07<00:00,  4.84it/s]


found: 0.94

CPU times: user 8h 11min 22s, sys: 20min 44s, total: 8h 32min 7s
Wall time: 7h 14min 31s


In [7]:
embeddings

,image_id,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
0,0,0.040063,0.570188,-0.205856,-0.207147,-0.685156,0.875572,0.195525,0.263069,-0.065040,...,-0.054556,-0.408539,-0.021889,-0.510256,0.099593,0.378148,-0.282832,0.130747,-0.129671,0.143439
1,1,-0.231017,0.176313,0.341615,-0.343364,0.000140,0.071841,-0.101578,-0.705418,0.642879,...,0.357738,0.593474,0.244515,-0.143392,-0.100900,-0.164380,-0.514502,1.008251,-0.200839,-0.380208
2,2,-0.167542,0.041370,-0.097727,0.040258,-0.048385,-0.409488,-0.218555,-0.256703,-0.324521,...,-0.294237,0.239773,-0.184432,0.148067,0.111601,0.057123,-0.116048,0.622953,0.016619,-0.206443
3,3,-0.185815,0.233224,-0.220207,-0.164292,0.393449,0.338707,0.069729,-0.118172,-0.120118,...,-0.227938,0.228972,0.464048,-0.394989,0.091809,-0.206579,0.053853,0.171857,0.299081,0.341985
4,4,-0.070075,0.404768,-0.008263,0.457253,-0.185769,0.590136,0.592431,-0.248257,-0.056639,...,0.061216,0.476642,0.037116,0.096425,-0.220578,0.316117,-0.340564,1.100186,-0.184208,0.103563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69475,69475,-0.439497,0.024630,0.029694,0.313401,0.049096,0.196326,0.087322,0.011149,-0.195162,...,-0.122056,0.569884,0.877554,0.347739,-0.220037,-0.376298,-0.532710,0.017886,0.272139,0.416286
69476,69476,-0.336727,0.105042,-0.014149,-0.100525,-0.060500,-0.326088,0.016037,0.555359,0.188496,...,-0.092925,-0.141165,-0.001366,0.181614,0.395988,-0.218431,-0.261715,0.478720,-0.122022,-0.443649
69477,69477,-0.363973,0.260711,-0.059261,0.522644,0.079741,-0.041364,0.136542,-0.057749,0.121418,...,-0.130326,0.152554,0.655627,0.368892,-0.085061,-0.249023,-0.213025,0.777652,0.194381,0.108485
69478,69478,-0.068141,0.252226,-0.018986,0.265419,-0.407583,-0.146289,0.364292,-0.012511,-0.348367,...,-0.235141,0.199773,0.046873,0.079650,0.300423,-0.456441,-0.259283,0.432832,0.052929,0.273239


In [8]:
sizes['pixel_size_ratio'] = sizes['width'] / (sizes['height'] + 1e-3)
sizes['pixel_area'] = np.log1p(sizes['width'] * sizes['height'])

sizes.rename(columns={'width': 'pixel_width', 'height': 'pixel_height'}, inplace=True)

sizes

,image_id,pixel_width,pixel_height,pixel_size_ratio,pixel_area
0,0,300.0,200.0,1.499993,11.002117
1,1,300.0,198.0,1.515144,10.992066
2,2,300.0,204.0,1.470581,11.021919
3,3,300.0,397.0,0.755666,11.687727
4,4,300.0,225.0,1.333327,11.119898
...,...,...,...,...,...
69475,69475,300.0,235.0,1.276590,11.163382
69476,69476,300.0,235.0,1.276590,11.163382
69477,69477,300.0,151.0,1.986742,10.721084
69478,69478,300.0,209.0,1.435400,11.046133


In [9]:
sizes[sizes['pixel_width'].notna()]

,image_id,pixel_width,pixel_height,pixel_size_ratio,pixel_area
0,0,300.0,200.0,1.499993,11.002117
1,1,300.0,198.0,1.515144,10.992066
2,2,300.0,204.0,1.470581,11.021919
3,3,300.0,397.0,0.755666,11.687727
4,4,300.0,225.0,1.333327,11.119898
...,...,...,...,...,...
69475,69475,300.0,235.0,1.276590,11.163382
69476,69476,300.0,235.0,1.276590,11.163382
69477,69477,300.0,151.0,1.986742,10.721084
69478,69478,300.0,209.0,1.435400,11.046133


In [10]:
with open('image_embeddings.pickle', 'wb') as file:
    cloudpickle.dump(embeddings[range(512)].values, file)
    
sizes.to_csv('sizes.csv', index=False)

In [11]:
!rm pixels.csv